In [70]:
import pandas as pd
import re
import requests

#### Данные:

[Визиты](https://drive.google.com/file/d/1QosQQ4RRNR9rkL4t7sB707h2Uy0XfYJe/view?usp=drive_link) - тысяча записей с визитами

[Регистрации](https://drive.google.com/file/d/1AeQz0kaSgz0lxYSDtuNm36muhy5fRCzZ/view?usp=drive_link) - тысяча записей о первых регистрациях


In [152]:
visits_file_link = 'https://drive.google.com/file/d/1QosQQ4RRNR9rkL4t_7sB707h2Uy0XfYJe/view?usp=drive_link'
registrations_file_link = 'https://drive.google.com/file/d/1AeQz0kaSgz0lxYSDtuNm36muhy5fRCzZ/view?usp=drive_link'

In [154]:
def get_file_id(google_file_link):
    search_result = re.search(r'/d/([_\w]+)/', google_file_link)
    if search_result:
        return search_result.group(1)


def get_download_link(file_id):
    return f'https://drive.google.com/uc?export=download&id={file_id}'

In [164]:
visits_link = get_download_link(visits_file_id)
visits_link

'https://drive.google.com/uc?export=download&id=1QosQQ4RRNR9rkL4t_7sB707h2Uy0XfYJe'

In [158]:
with open('visits_1k.csv', 'wb') as file:
    file.write(response.content)

In [160]:
visits_link = get_download_link(get_file_id(visits_file_link))
registrations_file = get_download_link(get_file_id(registrayions_file_link))

In [162]:
registrayions_link

'https://drive.google.com/file/d/1AeQz0kaSgz0lxYSDtuNm36muhy5fRCzZ/view?usp=drive_link'